In [37]:
import numpy as np
import nibabel as nib
import nilearn
import nilearn.image
import os

In [38]:
# load neuroquery map
img = nib.load("music_functional_connectivity.nii.gz")
img.shape #(46, 55, 46)
img.affine

array([[   4.,    0.,    0.,  -90.],
       [   0.,    4.,    0., -126.],
       [   0.,    0.,    4.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [39]:
# load the functional data
sub_peek = '0001'
func = f'../shared/ds003097/derivatives/fmriprep/sub-{sub_peek}/func/sub-{sub_peek}_task-moviewatching_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
func_img = nib.load(func)

In [40]:
#resample to the same space
resampled = nilearn.image.resample_img(img, target_affine=func_img.affine, target_shape= (60, 77, 65))
resampled

In [41]:
import ipyvolume as ipv
array_mask = resampled.get_fdata() > 6.5
ipv.quickvolshow(array_mask)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)


In [42]:
## be careful, running this cell twice will give incorrect results!

# make boolean, threshold on 6.5
resampled.get_fdata()[resampled.get_fdata() < 6.5] = 0
resampled.get_fdata()[resampled.get_fdata() > 6.5] = 1

# sanity check
np.unique(resampled.get_fdata())

array([0., 1.])

In [43]:
final_img = nib.Nifti1Image(resampled.get_fdata(), resampled.affine, resampled.header)
nib.save(final_img, 'final_mask.nii.gz')

In [44]:
# apply mask to functional data
from nilearn.input_data import NiftiMasker

func_file = func
mask_file = 'final_mask.nii.gz'

masker = NiftiMasker(mask_img=mask_file)

func_data = masker.fit_transform(func_file)

In [45]:
func_data

array([[31505.65 , 32804.855, 37233.28 , ..., 22785.076, 29876.473,
        29123.264],
       [31777.691, 32373.115, 37552.26 , ..., 22278.947, 29481.477,
        29060.377],
       [31567.66 , 32282.082, 37078.734, ..., 22238.963, 29706.92 ,
        28748.23 ],
       ...,
       [32367.734, 31948.363, 37147.734, ..., 22882.15 , 28745.475,
        29669.377],
       [32352.63 , 31625.865, 37094.21 , ..., 23288.795, 28722.404,
        29849.08 ],
       [32987.203, 31944.975, 37359.883, ..., 22808.258, 28740.785,
        29646.963]], dtype=float32)

In [47]:
np.average(func_data, axis=1) #time series averaged over 425 voxels for 1 participant

array([31480.63 , 31470.809, 31464.06 , 31531.045, 31618.418, 31528.172,
       31502.094, 31539.242, 31581.893, 31618.934, 31498.203, 31538.033,
       31567.33 , 31594.312, 31547.756, 31592.63 , 31543.172, 31588.723,
       31542.02 , 31569.93 , 31546.1  , 31626.977, 31582.148, 31581.44 ,
       31548.014, 31603.582, 31670.674, 31614.777, 31568.832, 31583.002,
       31589.186, 31637.508, 31703.613, 31562.479, 31554.494, 31632.32 ,
       31602.705, 31578.473, 31557.7  , 31670.096, 31545.273, 31599.125,
       31554.492, 31533.148, 31533.666, 31659.389, 31498.215, 31771.486,
       31556.758, 31529.04 , 31547.525, 31624.38 , 31593.059, 31576.889,
       31649.518, 31637.068, 31595.104, 31461.49 , 31546.115, 31509.936,
       31463.08 , 31536.854, 31611.102, 31606.771, 31622.088, 31526.941,
       31474.63 , 31547.057, 31543.43 , 31564.826, 31554.748, 31581.336,
       31563.23 , 31477.008, 31440.338, 31441.727, 31383.016, 31399.502,
       31449.734, 31425.582, 31389.445, 31396.574, 

In [48]:
# sanity check, should give the number of trs
len(np.average(func_data,axis=1))

290